# Frequencies in PWV rariation and comparison to Merra2

- author Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date 2025-03-03 :
- last update 2025-03-03
- last update : 2025-03-06  : test stationarity # p-value for a 2-sided t-test from scipy import stats
        2*(stats.t.sf( abs(tau_avg) ,dof=1000 ))
        https://stackoverflow.com/questions/35544233/fit-a-curve-to-a-histogram-in-python
        statsmodels
- Kernel @usdf **w_2025_50*
- Office emac : mamba_py311
- Home emac : base (conda)


**Goal** : Show correlation holo /Merra

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import os,glob

In [ ]:
#from pyzdcf import pyzdcf

In [ ]:
# where are stored the figures
pathfigs = "figsPWVFrequenciesAndMerra2"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
pathdata = "dataPWVFrequenciesAndMerra2"
if not os.path.exists(pathdata):
    os.makedirs(pathdata) 
datapath_input = os.path.join(pathdata,"timecurves") 
datapath_output = os.path.join(pathdata,"results") 
if not os.path.exists(datapath_input):
    os.makedirs(datapath_input) 
if not os.path.exists(datapath_output):
    os.makedirs(datapath_output) 

In [ ]:
ls ../SpectroSimulation/AnaMerra2/

In [ ]:
# where are stored the figures
inputpathm2residualsdata = "../SpectroSimulation/AnaMerra2/dataFitGPPerAtmosphereFromMerra2"
if not os.path.exists(inputpathm2residualsdata):
    #os.makedirs(pathdata) 
    raise Exception("Missing input Merra2 data on residuals")
else:
    filessearch_str = os.path.join(inputpathm2residualsdata ,"*.csv")
    filelist = glob.glob( filessearch_str)

In [ ]:
filename_residuals = {}
for tag in ["pwv", "ozone","vaod", "angstrom"]:
    for filename in filelist:
        if tag in filename:
            filename_residuals[tag] = filename
            break
filename_residuals  

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib.gridspec as gridspec

from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from astropy.coordinates.earth import EarthLocation
from datetime import datetime
from pytz import timezone

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit,least_squares


# new color correction model
import pickle
#from scipy.interpolate import RegularGridInterpolator

In [ ]:
from astropy.modeling import models, fitting

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time
from astropy.timeseries import LombScargle

from scipy.stats import t  # Loi de Student

In [ ]:
#from scipy.interpolate import interp

In [ ]:
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import ExpSineSquared
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import WhiteKernel
from sklearn.gaussian_process.kernels import ConstantKernel
from sklearn.gaussian_process import GaussianProcessRegressor

In [ ]:
# Remove to run faster the notebook
import ipywidgets as widgets
%matplotlib widget

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
import pandas as pd
pd.__version__

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

In [ ]:
YEAR = 365.25
MONTHS6 = YEAR/2.
MONTHS4 = YEAR/3.
QUARTER = YEAR/4. 
DAY = 1.
MONTH = YEAR/12.
WEEK = 7*DAY

In [ ]:
FIGXSIZE_1 = 14
FIGYSIZE_1 = 8

FIGXSIZE_0 = 14
FIGYSIZE_0 = 5

In [ ]:
def LombScargle_analysis(dates, values, ax ,mode = "logxlogy",title="LombScargle - Spectrum",
                    xlabel="frequency (day)$^{-1}$",ylabel="y-unit",label="Lomb Scargle", legendout = True, datecut = 0 ):
    # Centrer les données autour de la moyenne


    if datecut>0:
        index_selected = np.where(dates >= datecut)[0]
        dates = dates[index_selected]
        values = values[index_selected]

    
    values_centered = values - np.mean(values)
    

    # Nombre de points
    N = len(dates)

    # sigma
    sigma_x = np.sqrt(np.sum(values_centered**2)/N)
    
    # Intervalle d'échantillonnage (assume 1 jour entre chaque point)
    T = np.mean(np.diff(dates))  # Période d'échantillonnage

    # Fréquence de Nyquist (limite de Shannon)
    f_nyquist = 1 / (2 * T)
    

    freqs, power = LombScargle(dates, values_centered).autopower()
   
    ax.plot(freqs, power,'ob-' ,ms=5,label=label)

    if mode == "logxliny":
        ax.set_xscale("log")  # Définit l'axe X en échelle logarithmique
        ax.set_yscale("linear")  # Garde l'axe Y en échelle linéaire
    elif mode == "logxlogy":
        ax.set_xscale("log")  # Définit l'axe X en échelle logarithmique
        ax.set_yscale("log")  # Garde l'axe Y en échelle logarithmique
    elif mode == "linxlogy":
        ax.set_xscale("linear")  # Définit l'axe X en  échelle linéaire
        ax.set_yscale("log")  # Garde l'axe Y en échelle logarithmique
    elif mode == "linxliny":
        ax.set_xscale("linear")  # Définit l'axe X en échelle linéaire
        ax.set_yscale("linear")  # Garde l'axe Y en échelle linéaire
        
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    

    ax.axvline(1/YEAR, color='r', linestyle='-', label="Cycle : 365 days - 1 year")
    ax.axvline(1/MONTHS6, color='r', linestyle='--', label="Cycle : 182.6 days - 6 months")
    ax.axvline(1/MONTHS4, color='r', linestyle=':', label="Cycle : 121.7 days - 4 months")
    ax.axvline(1/QUARTER, color='r', linestyle='-.', label="Cycle : 91.3 days - 3 months")
    ax.axvline(1/MONTH, color='r', linestyle=':', label="Cycle : 30.4 days - 1 month")
    ax.axvline(1/WEEK, color='purple', linestyle='--', label="Cycle : 7 days - 1 week")
    ax.axvline(DAY, color='purple', linestyle='-', label="Cycle : 1 day ")
    ax.axvline(1./(0.5*DAY), color='purple', linestyle='-.', label="Cycle : 0.5 day ")

    #ax.axvline(f_nyquist, color='g', linestyle='--', label=f"Nyquist frequency({f_nyquist:.3f} cycles/days)")

    txtstr_sigma = "$\sigma_x$ = " + f" {sigma_x:0.3f}" 
    ax.text(0.01, 0.95, txtstr_sigma, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)
    

    if legendout:
        ax.legend(bbox_to_anchor=(1.05, 1.05),fontsize=12)
    else:
        ax.legend(fontsize=10,fancybox=True, framealpha=0.5)
        


## Configuration

In [ ]:
observing_location = EarthLocation.of_site('Rubin Observatory')
tz = timezone('America/Santiago')

## Merra2

In [ ]:
filename_m2 = "../SpectroMerra2/MerradataMerged/Merge_inst1_2d_asm_Nx_M2I1NXASM-2021-2024.csv"
if not os.path.isfile(filename_m2):
    #os.makedirs(pathdata) 
    raise Exception("Missing input Merra2 input file {filename_m2}")

In [ ]:
df_m = pd.read_csv(filename_m2,index_col=0)

In [ ]:
df_m["mjd"] = Time(pd.to_datetime(df_m.time.values)).mjd

In [ ]:
df_m.head()

In [ ]:
df_m

#### Add residuals from Gaussian process periodic fit

In [ ]:
full_filename_m2res = filename_residuals["pwv"]
print(full_filename_m2res)
df_m2res = pd.read_csv(full_filename_m2res,index_col=0)

In [ ]:
df_m2res

In [ ]:
df_m2 = pd.merge(df_m, df_m2res, on='mjd')

In [ ]:
df_m2.plot(x="mjd",y="res")

#### Build the Interpolation Function

In [ ]:
X_M2 = df_m2.mjd.values
Y_M2 = df_m2.TQV.values	
Y_M2res = df_m2.res.values	
func_M2interp = lambda x : np.interp(x,X_M2,Y_M2) 
func_M2resinterp = lambda x : np.interp(x,X_M2,Y_M2res) 

### Spectro Hologram data

In [ ]:
FLAG_WITHCOLLIMATOR = True
DATE_WITHCOLLIMATOR = 20230930
datetime_WITHCOLLIMATOR = convertNumToDatestr(DATE_WITHCOLLIMATOR)
datetime_WITHCOLLIMATOR = pd.to_datetime("2023-09-30 00:00:00.0+0000")
DATEMJD_WITHCOLLIMATOR = Time(datetime_WITHCOLLIMATOR).mjd

In [ ]:
version_results = "v5"
legendtag = {"v1" : "old v3.1.0",
            "v2" : "v3.1.0-PWV<10mm",
            "v3" : "v3.1.0-PWV<15mm",
            "v4" : "Auxtel holo v3.1.0",
            "v5" : "Auxtel holo v3.1.0 09/22 - 10/24"}

In [ ]:
atmfilenamesdict = {"v1" : "data/spectro/auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_testWithMaskedEdges_newBoundaries_newPolysRescaled_newFitBounds_adjustA1_lockedOrder2_removeThroughputTails_2.npy",
                    "v2" : "auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_PeekFinder.npy",
                    "v3" : "u_dagoret_auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_PeekFinder_20240924T161119Z.npy",
                    "v4" : "u_dagoret_auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_PeekFinder_20240924T161119Z_spectrfullextend.npy",
                    "v5" : "u_dagoret_auxtel_atmosphere_202209_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_No5SigmaClip_20241016T184601Z_spectrfullextended.npy"}

In [ ]:
atmfilename = atmfilenamesdict[version_results]
tag = legendtag[version_results] 

## Initialisation

### Read the file

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)

In [ ]:
df_spec['pwv_m2'] = df_spec["ex_mjd"].apply(func_M2interp)
df_spec['pwvres_m2'] = df_spec["ex_mjd"].apply(func_M2resinterp)
df_spec["dpwv"] =   df_spec["PWV [mm]_x"] - df_spec['pwv_m2']

### Remove spectra with red filter

In [ ]:
df_spec['FILTER'].unique()

In [ ]:
FLAG_REMOVE_FILTERS = True
if FLAG_REMOVE_FILTERS:
    df_spec=df_spec[df_spec["FILTER"] == 'empty']
    df_spec.reset_index(inplace=True)  

### Define if a target is faint or bright

In [ ]:
def IsFaint(row):
    List_Of_Faint_targets = ['Feige110','HD074000','HD115169','HD031128','HD200654','HD167060','HD009051','HD142331','HD160617','HD111980']
    List_Of_faint_selected = List_Of_Faint_targets[:10]
    if row["TARGET"] in List_Of_faint_selected:
        return True
    else:
        return False

In [ ]:
df_spec["isFaint"] = df_spec.apply(IsFaint,axis=1)

### Compute NightObs

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
if FLAG_WITHCOLLIMATOR:
    df_spec = df_spec[df_spec["nightObs"]> DATE_WITHCOLLIMATOR]

## Apply Quality selection

In [ ]:
fig,ax = plt.subplots(1,1)
df_spec["CHI2_FIT"].hist(bins=50,ax=ax,range=(0,200))
ax.set_yscale("log")

### Add the Time in pd.datetime

#### UTC

In [ ]:
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])

In [ ]:
DT = pd.Timedelta(minutes=7*24*60)
TMIN  = df_spec["Time"].min()-DT
TMAX  = df_spec["Time"].max()+DT

TMINMJD = Time(TMIN).mjd
TMAXMJD = Time(TMAX).mjd

### Compute relative time to Mid-night

In [ ]:
def GetTimeToMidNight(row):
    observing_time = Time(row['DATE-OBS'], scale='utc', location=observing_location)

    # time at the location , either before or after midnight
    local_time =  observing_time.to_datetime(timezone=tz)

    # take time independent  of any location now
    local_time_new = datetime(local_time.year,local_time.month,local_time.day,local_time.hour,local_time.minute,local_time.second)
    local_time_midnight = datetime(local_time_new.year,local_time_new.month,local_time_new.day)
    dt_hour = (local_time_new -local_time_midnight).seconds/3600.

    # we took the previous night mid-night , must subtract 24H
    if dt_hour > 12.:
        dt_hour_new = (dt_hour - 24.)
    else:
        dt_hour_new = dt_hour
        
    return dt_hour_new

In [ ]:
df_spec["dt_midnight"] = df_spec.apply(GetTimeToMidNight,axis=1)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,4))
df_spec["dt_midnight"].hist(bins=48,range=(-12,12),ax=ax,facecolor="blue") 
ax.set_xlabel("time relative to midnight (hour)")
ax.set_title("Observation time")

### Compute Date relative to January

In [ ]:
def GetDateToMidJanuary(row):
    observing_time = Time(row['DATE-OBS'], scale='utc', location=observing_location)

    # time at the location , either before or after midnight
    local_time =  observing_time.to_datetime(timezone=tz)

    # take time independent  of any location now
    local_time_new = datetime(2024,local_time.month,local_time.day,local_time.hour,local_time.minute,local_time.second)
           
    return pd.to_datetime(local_time_new)

In [ ]:
#df_spec["Time_january"] = df_spec.apply(GetDateToMidJanuary,axis=1)

In [ ]:
def GetDateToMidJanuaryAndYear(row):
    observing_time = Time(row['DATE-OBS'], scale='utc', location=observing_location)

    # time at the location , either before or after midnight
    local_time =  observing_time.to_datetime(timezone=tz)

    # take time independent  of any location now
    local_time_new = datetime(2024,local_time.month,local_time.day,local_time.hour,local_time.minute,local_time.second)
           
    return pd.to_datetime(local_time_new),local_time.year 

In [ ]:
df_spec[["Time_january","Year"]] = df_spec.apply(GetDateToMidJanuaryAndYear,axis=1,result_type="expand")

In [ ]:
df_spec[["Time_january","Year"]]

## Compute night boundaries

In [ ]:
def GetNightBoundariesDict(df_spec):
    """
    input:
      df_spec the dataframe for spectroscopy summary results
    output:
      the dict of night boudaries
    """
    
    Dt = pd.Timedelta(minutes=30)
    d = {}
    list_of_nightobs = df_spec["nightObs"].unique()
    for nightobs in list_of_nightobs:
        sel_flag = df_spec["nightObs"]== nightobs
        df_night = df_spec[sel_flag]
        tmin = df_night["Time"].min()-Dt
        tmax = df_night["Time"].max()+Dt
        d[nightobs] = (tmin,tmax)
    return d

In [ ]:
dn = GetNightBoundariesDict(df_spec)

## Plot all data

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax  = axs
leg=ax.get_legend()

ax.set_xlim(TMIN,TMAX) 
#df_m.plot(x="Time",y="PShP",ax=ax,marker=".",c="b",lw=0.5,label="Merra2",ms=1,legend=leg)   
df_spec.plot(x="Time",y="OUTPRESS",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)
ax.set_ylabel("OUTPRESS")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Pressure (before quality cuts)")
ax.legend()

if not FLAG_WITHCOLLIMATOR:
    ax.axvspan(TMIN,datetime_WITHCOLLIMATOR, color='yellow', alpha=0.1)


for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)


#ax.set_ylim(0.,15.)

figname =f"{pathfigs}/pressure_allpoints_allnights_nocuts"+figtype
plt.savefig(figname)
plt.show()


In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
fig,axs = plt.subplots(1,1,figsize=(18,8))
ax  = axs
leg=ax.get_legend()

ax.set_xlim(TMIN,TMAX)   

df_m.plot(x="Time",y="TQV",ax=ax,marker=".",c="b",lw=1,ms=1,ls="-",label="Merra2",legend=leg)   
df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)
ax.set_ylabel("PWV [mm]_x")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Precipitable water vapor measured by holo vs time (before quality cuts)")
ax.legend()

if not FLAG_WITHCOLLIMATOR:
    ax.axvspan(TMIN,datetime_WITHCOLLIMATOR, color='yellow', alpha=0.1)


for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)


ax.set_ylim(0.,15.)

figname =f"{pathfigs}/pwv_allpoints_allnights_nocuts"+figtype
plt.savefig(figname)
plt.show()


## Apply Quality selection cuts

In [ ]:
def getSelectionCut(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.,vaodmax=0.1):
    #cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.75) &  (df_spec["D2CCD"]<187.75) &  (df_spec["VAOD_x"]<0.1) & \
    (df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) & \
    (df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & \
    (df_spec["AIRMASS"] < 2)
    return cut

In [ ]:
def getSelectionCutNoPolar(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.,vaodmax=0.1):
    #cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.75) &  (df_spec["D2CCD"]<187.75) &  (df_spec["VAOD_x"]<0.1) & \
    (df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) & \
    (df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & (df_spec["TARGET"] != "HD185975") & \
    (df_spec["AIRMASS"] < 2)
    return cut

In [ ]:
def getSelectionCutWithPolar(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.,vaodmax=0.1):
    #cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.75) &  (df_spec["D2CCD"]<187.75) &  (df_spec["VAOD_x"]<0.1) & \
    (df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) & \
    (df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & (df_spec["TARGET"] == "HD185975") & \
    (df_spec["AIRMASS"] < 2)
    return cut

In [ ]:
cut = getSelectionCut(df_spec) 
cut_nopolar = getSelectionCutNoPolar(df_spec) 
cut_nopolar_bright = getSelectionCutNoPolar(df_spec) & (~df_spec["isFaint"])
cut_nopolar_faint = getSelectionCutNoPolar(df_spec) & (df_spec["isFaint"])
cut_wthpolar = getSelectionCutWithPolar(df_spec)

In [ ]:
df_spec_sel = df_spec[cut]
df_spec_np = df_spec[cut_nopolar] 
df_spec_np_b = df_spec[cut_nopolar_bright]
df_spec_np_f = df_spec[cut_nopolar_faint]
df_spec_wp = df_spec[cut_wthpolar]

In [ ]:
print("Total number of Spectra          : ",len(df_spec))
print("Number of selected Spectra       : ",len(df_spec_sel))
print("Number of selected Polars        : ",len(df_spec_wp))
print("Number of selected Non-Polars    : ",len(df_spec_np))
print("Number of selected Non-Polars Bright : ",len(df_spec_np_b))
print("Number of selected Non-Polars Faint  : ",len(df_spec_np_f))

In [ ]:
df_spec_sel.reset_index(drop=True,inplace=True)
df_spec_np.reset_index(drop=True,inplace=True)
df_spec_wp.reset_index(drop=True,inplace=True) 
df_spec_np_b.reset_index(drop=True,inplace=True)
df_spec_np_f.reset_index(drop=True,inplace=True)

In [ ]:
#List_Of_Faint_targets = ['Feige110','HD074000','HD115169','HD031128','HD200654','HD167060','HD009051','HD142331','HD160617','HD111980']
print("Polar            :",len(df_spec_wp["TARGET"].unique()),"\t", df_spec_wp["TARGET"].unique()) 
print("Non Polar        :",len(df_spec_np["TARGET"].unique()),"\t" ,df_spec_np["TARGET"].unique())
print("Non Polar Bright :",len(df_spec_np_b["TARGET"].unique()),"\t" ,df_spec_np_b["TARGET"].unique())
print("Non Polar Faint  :",len(df_spec_np_f["TARGET"].unique()),"\t",df_spec_np_f["TARGET"].unique())

## Recompute night boundaries

In [ ]:
#dn = GetNightBoundariesDict(df_spec_sel)

## Plot all data

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%d")

fig,axs = plt.subplots(1,1,figsize=(14,6),layout='constrained')
ax  = axs
leg=ax.get_legend()

ax.set_xlim(TMIN,TMAX) 
#df_m.plot(x="Time",y="TQV",ax=ax,marker=".",c="b",lw=0.5,ms=1,label="Merra2",legend=leg)     

df_m.plot(x="Time",y="TQV",ax=ax,marker=".",c="b",lw=1,ms=1,ls="-",label="Merra2",legend=leg)  
df_spec_sel.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)

#df_spec_sel[df_spec_sel.Year==2022].plot(x="Time_january",y="PWV [mm]_x",ax=ax,marker='+',c="g",lw=0.0,grid=True,label="2022",legend=leg,ms=5,alpha=1.)
#df_spec_sel[df_spec_sel.Year==2023].plot(x="Time_january",y="PWV [mm]_x",ax=ax,marker='+',c="b",lw=0.0,grid=True,label="2023",legend=leg,ms=5,alpha=1.)
#df_spec_sel[df_spec_sel.Year==2024].plot(x="Time_january",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label="2024",legend=leg,ms=5,alpha=1.)

ax.set_ylabel("PWV [mm]_x")
ax.set_xlabel("date")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Precipitable water vapor measured by holo",fontweight="bold")
ax.legend()

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

ax.axvspan(TMIN,datetime_WITHCOLLIMATOR, color='yellow', alpha=0.1)

plt.tight_layout()
 
ax.set_ylim(0.,15.)
#ax.set_xlim(TMIN,TMAX)

figname =f"{pathfigs}/pwvholoM2_allpoints_allnights"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
x_mjd = np.arange(TMINMJD,TMAXMJD,1/24/60)
y_mjd = func_M2interp(x_mjd)

In [ ]:
fig,axs = plt.subplots(1,1,figsize=(14,6),layout='constrained')
ax  = axs
leg=ax.get_legend()


df_m.plot(x="mjd",y="TQV",ax=ax,marker=".",c="b",lw=0,ms=5,label="Merra2",legend=leg)  

ax.plot(x_mjd,y_mjd,'-k',lw=0.5)

ax.set_ylabel("PWV [mm]_x")
ax.set_xlabel("date")
#ax.xaxis.set_major_formatter(date_form)
ax.set_title("Precipitable water vapor measured by Mera2",fontweight="bold")
ax.legend()

#for key, tt in dn.items():
#    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)


plt.tight_layout()
 
ax.set_ylim(0.,25.)
ax.set_xlim(TMINMJD,TMAXMJD)


plt.show()

In [ ]:
fig,axs = plt.subplots(1,1,figsize=(14,6),layout='constrained')
ax  = axs
leg=ax.get_legend()

#df_m.plot(x="mjd",y="TQV",ax=ax,marker=".",c="b",lw=1,ms=1,ls="-",label="Merra2",legend=leg)  
df_m.plot(x="mjd",y="TQV",ax=ax,marker=".",c="b",lw=0,ms=3,label="Merra2",legend=leg)  
df_spec_sel.plot(x="ex_mjd",y="PWV [mm]_x",ax=ax,marker='o',ms=3,c="r",lw=0.0,grid=True,label=tag,legend=leg)


ax.set_ylabel("PWV [mm]_x")
ax.set_xlabel("date")
#ax.xaxis.set_major_formatter(date_form)
ax.set_title("Precipitable water vapor measured by holo",fontweight="bold")
ax.legend()

#for key, tt in dn.items():
#    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

ax.axvspan(TMINMJD,DATEMJD_WITHCOLLIMATOR , color='yellow', alpha=0.1)

plt.tight_layout()
 
ax.set_ylim(0.,20.)
ax.set_xlim(TMINMJD,TMAXMJD)

figname =f"{pathfigs}/pwvholoM2_allpoints_allnights_mjd"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
fig = plt.figure(figsize=(16,10),layout='constrained')

gs = gridspec.GridSpec(2,1,height_ratios=[2, 1])
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1], sharex=ax1)

leg1=ax1.get_legend()
leg2=ax2.get_legend()

df_m.plot(x="mjd",y="TQV",ax=ax1,marker=".",c="b",lw=0,ms=5,label="Merra2",legend=leg1)  
#df_spec_sel.plot(x="ex_mjd",y="PWV [mm]_x",ax=ax1,marker='o',ms=5,c="r",lw=0.0,grid=True,label="data",legend=leg1)

df_spec_wp.plot(x="ex_mjd",y="PWV [mm]_x",ax=ax1,marker='o',ms=5,c="r",lw=0.0,grid=True,label="with polar",legend=leg1)
df_spec_np_b.plot(x="ex_mjd",y="PWV [mm]_x",ax=ax1,marker='o',ms=5,c="purple",lw=0.0,grid=True,label="non polar bright",legend=leg1)
df_spec_np_f.plot(x="ex_mjd",y="PWV [mm]_x",ax=ax1,marker='o',ms=5,c="green",lw=0.0,grid=True,label="non polar faint",legend=leg1)


ax1.set_ylabel("PWV [mm]_x")

#ax.xaxis.set_major_formatter(date_form)
ax1.set_title("Precipitable water vapor measured by holo",fontweight="bold")
ax1.legend()

ax1.set_ylim(0.,15.)
ax1.set_xlim(TMINMJD,TMAXMJD)
ax1.axvspan(TMINMJD,DATEMJD_WITHCOLLIMATOR , color='yellow', alpha=0.1)

#df_spec_sel.plot(x="ex_mjd",y="dpwv",ax=ax2,marker='o',ms=5,c="r",lw=0.0,grid=True,label="data- merra2",legend=leg2)
df_spec_wp.plot(x="ex_mjd",y="dpwv",ax=ax2,marker='o',ms=4,c="r",lw=0.0,grid=True,label="with polar",legend=leg2)
df_spec_np_b.plot(x="ex_mjd",y="dpwv",ax=ax2,marker='o',ms=4,c="purple",lw=0.0,grid=True,label="non polar bright",legend=leg2)
df_spec_np_f.plot(x="ex_mjd",y="dpwv",ax=ax2,marker='o',ms=4,c="green",lw=0.0,grid=True,label="non polar faint",legend=leg2)

ax2.set_ylabel("$\Delta PWV$ (mm)")
ax2.set_xlabel("date")
ax2.axvspan(TMINMJD,DATEMJD_WITHCOLLIMATOR , color='yellow', alpha=0.1)
ax2.legend()


plt.tight_layout()
#figname =f"{pathfigs}/pwvholoM2_allpoints_allnights_mjd"+figtype
#plt.savefig(figname)
plt.show()

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(14,6),layout='constrained')
ax1,ax2  = axs
leg=ax.get_legend()

#sc = ax.scatter(data=df_spec_sel, x='PWV [mm]_x', y='dpwv', c='ex_mjd', marker="+",s=30,cmap='rainbow')
sc1 = ax1.scatter(data=df_spec_sel, x='pwv_m2', y='dpwv', c='ex_mjd', marker="+",s=30,cmap='rainbow')

ax1.set_title("$\Delta PWV$ vs PWV")
ax1.set_xlabel("PWV (mm) (Merra)")
ax1.set_ylabel("$\Delta$ PWV (Merra) (mm)")


#
sc1 = ax2.scatter(data=df_spec_sel, x='pwv_m2', y='dpwv', c='ex_mjd', marker="+",s=30,cmap='rainbow')

ax2.set_title("$\Delta PWV$ vs PWV")
ax2.set_xlabel("PWV (mm) (AUXTEL)")
ax2.set_ylabel("$\Delta$ PWV (mm)")
sc2 = ax2.scatter(data=df_spec_sel, x='PWV [mm]_x', y='dpwv', c='ex_mjd', marker="+",s=30,cmap='rainbow')


cbar = plt.colorbar(sc1)
plt.show()

### Fit straight line to find a link between 

### The two functions 

In [ ]:
# 🌟 Modèle linéaire : y = ax + b
def linearmodel(x, a, b):
    return a * x + b

In [ ]:
#def error_propagation(x, popt, pcov):
#    """Calcule l'incertitude sur y(x) en tenant compte des erreurs sur a et b."""
#    g = np.array([x, 1])  # Gradient de f(x) = ax + b
#    sigma_y2 = g.T @ pcov @ g  # Variance de y
#    return np.sqrt(sigma_y2)  # Écart-type

def error_propagation(x, popt, pcov):
    """Calcule l'incertitude sur y(x) pour un x scalaire ou un tableau numpy."""
    x = np.atleast_1d(x)  # S'assure que x est un tableau
    g = np.vstack([x, np.ones_like(x)])  # Matrice (2, N) avec les gradients [x, 1] en colonne
    sigma_y2 = np.einsum('ij,jk,ik->i', g.T, pcov, g.T)  # Produit matriciel pour chaque x
    return np.sqrt(sigma_y2)  # Retourne l'écart-type de y(x)

def ndof(x,popt):
    dof = len(x) - len(popt) 
    return dof
    

def error_propagation_with_IC(x, popt, pcov, dof, confidence=0.95):
    """Calcule l'intervalle de confiance à `confidence` % sur y(x)."""
    x = np.atleast_1d(x)  
    g = np.vstack([x, np.ones_like(x)])  # Matrice (2, N) des gradients [x, 1]
    sigma_y2 = np.einsum('ij,jk,ik->i', g.T, pcov, g.T)  # Variance de y(x)
    sigma_y = np.sqrt(sigma_y2)  # Écart-type
    
    # Facteur de Student pour l'intervalle de confiance
    alpha = 1 - confidence
    t_factor = t.ppf(1 - alpha / 2, dof)  # Quantile de Student

    return sigma_y * t_factor  # Intervalle de confiance autour de y(x)

In [ ]:
def param_label(popt):
    txt_param1 = f"slope = {popt[0]:.3f}"
    txt_param2 = f"intercept = {popt[1]:.3f}"
    txtstr = "\n".join([txt_param1,txt_param2])
    return txtstr

In [ ]:
XMIN = 0.
XMAX = 15.
xfit = np.linspace(XMIN,XMAX,500)

# define a model for a line
line_orig = models.Linear1D(slope=1.0, intercept=0.5)

# initialize a linear fitter
fit = fitting.LinearLSQFitter()

# initialize a linear model
line_init = models.Linear1D()
line_init1 = models.Linear1D()
line_init2 = models.Linear1D()
line_init3 = models.Linear1D()

In [ ]:
# fit the data with the fitter

x = df_spec_sel["pwv_m2"].values
y = df_spec_sel["PWV [mm]_x"].values
fitted_line = fit(line_init, x, y)


#popt, pcov = curve_fit(linearmodel, x, y, sigma=np.ones_like(y) * 1.0, absolute_sigma = True)
popt, pcov = curve_fit(linearmodel, x, y, sigma=np.ones_like(y) * 1.0, p0 = [1, 0], bounds=([0.99, -np.inf], [1.01, np.inf]),absolute_sigma = True)
perr = np.sqrt(np.diag(pcov))
dof = ndof(x,popt)

txtstr = param_label(popt)

x1 = df_spec_wp["pwv_m2"].values
y1 = df_spec_wp["PWV [mm]_x"].values
fitted_line1 = fit(line_init1, x1, y1)
#popt1, pcov1 = curve_fit(linearmodel, x1, y1)
popt1, pcov1 = curve_fit(linearmodel, x1, y1, sigma=np.ones_like(y1) * 1.0, p0 = [1, 0], bounds=([0.99, -np.inf], [1.01, np.inf]),absolute_sigma = True)
perr1 = np.sqrt(np.diag(pcov1))
dof1 = ndof(x1,popt1)

txtstr1 = param_label(popt1)

x2 = df_spec_np_b["pwv_m2"].values
y2 = df_spec_np_b["PWV [mm]_x"].values
fitted_line2 = fit(line_init2, x2, y2)
#popt2, pcov2 = curve_fit(linearmodel, x2, y2)
popt2, pcov2 = curve_fit(linearmodel, x2, y2,sigma=np.ones_like(y2) * 1.0, p0 = [1, 0], bounds=([0.99, -np.inf], [1.01, np.inf]),absolute_sigma = True)
perr2 = np.sqrt(np.diag(pcov2))
dof2 = ndof(x2,popt2)

txtstr2 = param_label(popt2)

x3 = df_spec_np_f["pwv_m2"].values
y3 = df_spec_np_f["PWV [mm]_x"].values
fitted_line3 = fit(line_init3, x3, y3)
#popt3, pcov3 = curve_fit(linearmodel, x3, y3)
popt3, pcov3 = curve_fit(linearmodel, x3, y3,sigma=np.ones_like(y3) * 1.0, p0 = [1, 0], bounds=([0.99, -np.inf], [1.01, np.inf]),absolute_sigma = True)
perr3 = np.sqrt(np.diag(pcov3))
dof3 = ndof(x3,popt3)

txtstr3 = param_label(popt3)

In [ ]:
pcov

In [ ]:
fig,axs = plt.subplots(1,1,figsize=(8,8),layout='constrained')
ax  = axs
leg=ax.get_legend()

#df_spec_sel.plot(x="pwv_m2",y="PWV [mm]_x",ax=ax,marker='o',ms=5,c="grey",lw=0.0,grid=True,label="",legend=leg)
df_spec_wp.plot(x="pwv_m2",y="PWV [mm]_x",ax=ax,marker='o',ms=5,c="r",lw=0.0,grid=True,label="with polar",legend=leg)
df_spec_np_b.plot(x="pwv_m2",y="PWV [mm]_x",ax=ax,marker='o',ms=5,c="purple",lw=0.0,grid=True,label="non polar bright",legend=leg)
df_spec_np_f.plot(x="pwv_m2",y="PWV [mm]_x",ax=ax,marker='o',ms=5,c="green",lw=0.0,grid=True,label="non polar faint",legend=leg)

ax.set_aspect("equal")
ax.set_xlim(0.,15.)
ax.set_ylim(0.,15.)

yfit = linearmodel(xfit,*popt)
#yfit_err = error_propagation(xfit, popt, pcov)
yfit_err =  error_propagation_with_IC(xfit, popt, pcov, dof, confidence=0.95)

ax.plot(xfit,yfit,"-k",lw=3)
ax.fill_between(xfit, yfit- yfit_err, yfit+ yfit_err, color='grey', alpha=0.2, label="confidence belt 95% CL")

ax.set_ylabel("PWV - Auxtel (mm)")
ax.set_xlabel("PWV - Merra2 (mm)")

ax.text(0.1, 0.95, txtstr, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)


ax.legend(loc="upper right")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 🔹 Création de la figure avec 3 subplots
fig = plt.figure(figsize=(18,6))

# 🔹 Utilisation de gridspec pour gérer l'espace
gs = fig.add_gridspec(2, 3, height_ratios=[0.95, 0.05])  # 95% pour les subplots, 5% pour la colorbar

# 🔹 Subplots principaux (en haut, dans la première ligne)
ax1 = fig.add_subplot(gs[0, 0])  # Premier subplot
ax2 = fig.add_subplot(gs[0, 1],sharex=ax1, sharey=ax1)  # Deuxième subplot
ax3 = fig.add_subplot(gs[0, 2],sharex=ax1, sharey=ax1)  # Troisième subplot


# 🔹 Tracé des graphiques (exemple de scatter plots)
leg1=ax1.get_legend()
leg2=ax2.get_legend()
leg3=ax3.get_legend()


#df_spec_wp.plot(x="pwv_m2",y="PWV [mm]_x",ax=ax1,marker='o',ms=3,c="r",lw=0.0,grid=True,label="with polar",legend=leg1)
sc1 = ax1.scatter(data=df_spec_wp, x='pwv_m2', y='PWV [mm]_x', c='ex_mjd', marker="+",s=30,cmap='rainbow',label="polar")

#df_spec_np_b.plot(x="pwv_m2",y="PWV [mm]_x",ax=ax2,marker='o',ms=3,c="purple",lw=0.0,grid=True,label="non polar bright",legend=leg2)
sc2 = ax2.scatter(data=df_spec_np_b, x='pwv_m2', y='PWV [mm]_x', c='ex_mjd', marker="+",s=30,cmap='rainbow',label="non-polar bright")

#df_spec_np_f.plot(x="pwv_m2",y="PWV [mm]_x",ax=ax3,marker='o',ms=3,c="green",lw=0.0,grid=True,label="non polar faint",legend=leg3)
sc3 = ax3.scatter(data=df_spec_np_f, x='pwv_m2', y='PWV [mm]_x', c='ex_mjd', marker="+",s=30,cmap='rainbow',label="non-polar faint")


ax1.set_aspect("equal")
ax2.set_aspect("equal")
ax3.set_aspect("equal")

ax1.set_xlim(0.,15.)
ax1.set_ylim(0.,15.)
ax1.text(0.1, 0.95, txtstr1, transform=ax1.transAxes, fontsize=16,verticalalignment='top', bbox=props)

yfit1 = linearmodel(xfit,*popt1)
yfit_err1 =  error_propagation_with_IC(xfit, popt1, pcov1, dof1, confidence=0.95)
ax1.plot(xfit,yfit1,"-k",lw=3)
ax1.fill_between(xfit, yfit1- yfit_err1, yfit1+ yfit_err1, color='grey', alpha=0.2,label="confidence belt 95% CL")



ax2.text(0.1, 0.95, txtstr2, transform=ax2.transAxes, fontsize=16,verticalalignment='top', bbox=props)
#ax2.plot(xfit,fitted_line2(xfit),"-k",lw=3)
yfit2 = linearmodel(xfit,*popt2)
yfit_err2 =  error_propagation_with_IC(xfit, popt2, pcov2, dof2, confidence=0.95)
ax2.plot(xfit,yfit2,"-k",lw=3)
ax2.fill_between(xfit, yfit2 - yfit_err2, yfit2 + yfit_err2, color='grey', alpha=0.2,label="confidence belt 95% CL")

ax3.text(0.1, 0.95, txtstr3, transform=ax3.transAxes, fontsize=16,verticalalignment='top', bbox=props)
#ax3.plot(xfit,fitted_line3(xfit),"-k",lw=3)
yfit3 = linearmodel(xfit,*popt3)
yfit_err3 =  error_propagation_with_IC(xfit, popt3, pcov3, dof3, confidence=0.95)
ax3.plot(xfit,yfit3,"-k",lw=3)
ax3.fill_between(xfit, yfit3 - yfit_err3, yfit3 + yfit_err3, color='grey', alpha=0.2,label="confidence belt 95% CL")



ax1.legend()
ax2.legend()
ax3.legend()

ax1.set_ylabel("PWV - Auxtel (mm)")
ax1.set_xlabel("PWV - Merra2 (mm)")
ax2.set_xlabel("PWV - Merra2 (mm)")
ax3.set_xlabel("PWV - Merra2 (mm)")
ax1.plot([0,15],[0,15])
ax2.plot([0,15],[0,15])
ax3.plot([0,15],[0,15])

# 🔹 Mise en forme et personnalisation des axes
ax1.set_aspect("equal")
ax2.set_aspect("equal")
ax3.set_aspect("equal")

ax1.set_xlim(0.,15.)
ax1.set_ylim(0.,15.)

# 🔹 Ajout de la colorbar sous les axes (dans la deuxième ligne de gridspec)
ax_cbar = fig.add_subplot(gs[1, :])  # Colorbar occupe toute la largeur de la deuxième ligne


fig.colorbar(sc1, cax=ax_cbar, orientation='horizontal')
cbar.set_label('date (MJD)', fontsize=12)
#cbar.ax.yaxis.set_label_position('center')  # Déplacer le label à droite si nécessaire

plt.suptitle("PWV Auxtel vs PWV Merra2")
plt.tight_layout()

# 🔹 Affichage du graphique
plt.show()


In [ ]:
cut = df_spec_np_b["pwv_m2"]>14
df_spec_np_b[cut]

In [ ]:
all_selected_nights = df_spec_sel["nightObs"].unique()

In [ ]:
#all_selected_nights 

## LombScargle

In [ ]:
x = df_spec["ex_mjd"].values
y = df_spec["PWV [mm]_x"].values

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
LombScargle_analysis(x,y, ax=ax ,mode= "logxliny",
                 title = "LombScargle : PWV residuals",
                 xlabel="frequency (days$^{-1}$)",
                 ylabel=" ",
                 label="Merra2 PWV")
figname =f"{pathfigs}/pwv_finalres_LombScargle_merra2"+figtype
fig.savefig(figname)
plt.show()

## Define Kernels

In [ ]:
# Tendance long terme
long_term_trend_kernel = ConstantKernel(.5, (0.0, 10.0)) * RBF(length_scale=365.0)

periodic_1year_kernel =  ConstantKernel(3.0, (0.1, 10.0)) * ExpSineSquared(length_scale= 10*YEAR, periodicity= YEAR,
                                                                           length_scale_bounds="fixed",periodicity_bounds="fixed")
periodic_6months_kernel = ConstantKernel(2.5, (0.1, 10.0)) * ExpSineSquared(length_scale= 20*MONTHS6,periodicity=MONTHS6,
                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed") 
periodic_3months_kernel = ConstantKernel(2.5, (0.1, 10.0)) * ExpSineSquared(length_scale= 40*QUARTER, periodicity=QUARTER,
                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed")
periodic_4months_kernel = ConstantKernel(2.5, (0.1, 10.0)) * ExpSineSquared(length_scale= 30*MONTHS4, periodicity=MONTHS4,
                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed")

seasonal_enveloppe = RBF(length_scale=YEAR, length_scale_bounds=(0.5*YEAR, 10*YEAR))

# Saisonnalité multi-échelle
seasonal_kernel = (
    #seasonal_enveloppe * ( periodic_1year_kernel + periodic_6months_kernel + periodic_3months_kernel)
    #seasonal_enveloppe * ( periodic_1year_kernel  + periodic_3months_kernel)
    #periodic_1year_kernel  + periodic_6months_kernel + periodic_3months_kernel
    #periodic_1year_kernel  + periodic_6months_kernel + periodic_4months_kernel + periodic_3months_kernel
    periodic_1year_kernel
)


# Petites fluctuations irrégulières
irregularities_kernel = ConstantKernel(1.0, (0.0, 10.0)) * RationalQuadratic(length_scale=DAY, alpha=1.0)

# Bruit et variations locales
#noise_kernel = ConstantKernel(1.0, (0., 10.0)) * RBF(length_scale=1.0) + WhiteKernel(noise_level=1.0**2)
noise_kernel = ConstantKernel(1.0, (0., 10.0)) * RBF(length_scale=1.0) 

# Kernel total
#full_kernel = long_term_trend_kernel + seasonal_kernel + irregularities_kernel + noise_kernel
#full_kernel = seasonal_kernel + irregularities_kernel
full_kernel = seasonal_kernel 


In [ ]:
pwv_kernel = full_kernel

In [ ]:
# subset of values choosen randomly to be fitted on 

x = df_spec["ex_mjd"].values
X = x.reshape(-1, 1)
y = df_spec["PWV [mm]_x"].values
y_median = np.median(y)
y_mean = np.mean(y)

In [ ]:
gaussian_process = GaussianProcessRegressor(kernel=pwv_kernel, normalize_y= True)
gaussian_process.fit(X, y)

In [ ]:
txtstr_kernel = f"{gaussian_process.kernel_}"
txtstr_kernel = "\n + ".join(txtstr_kernel.split("+ "))

In [ ]:
mjd_min = x.min()
mjd_max = x.max() + YEAR

mjd_zoom_start = Time("2024-01-01").mjd
mjd_zoom_stop = Time("2025-06-30").mjd

mjd_obs_start = x.min()
mjd_obs_stop = x.max()

In [ ]:
x_test = np.arange(start=mjd_min, stop=mjd_max,step=2)
X_test = x_test.reshape(-1,1)
mean_y_pred, std_y_pred = gaussian_process.predict(X_test, return_std=True)

In [ ]:
fig = plt.figure(figsize=(12,6),layout="constrained")
gs = GridSpec(2, 1,figure=fig)
#gs = GridSpec(1, 1,figure=fig)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
ax1.plot(x,y,'-o',color="k",ms=5,linestyle="dashed", label="M2 Measurements")
ax1.plot(x_test,mean_y_pred,color="tab:blue", lw=3 ,alpha=1.0, label="Gaussian process")
ax1.fill_between(
    X_test.ravel(),
    mean_y_pred - std_y_pred,
    mean_y_pred + std_y_pred,
    color="tab:blue",
    alpha=0.2,
)
ax1.legend()

ax1.set_ylabel("PWV (mm)")
ax1.set_xlabel("mjd")
ax1.set_title("Fit PWV with Gaussian process (subsample)")
ax1.text(0.1, 0.95, txtstr_kernel, transform=ax1.transAxes, fontsize=16,verticalalignment='top', bbox=props)

ax2.plot(x,y,'-o',color="k",ms=5,linestyle="dashed", label="M2 Measurements")
ax2.plot(x_test,mean_y_pred,color="tab:blue",lw=3 ,alpha=1.0, label="Gaussian process")
ax2.fill_between(
    X_test.ravel(),
    mean_y_pred - std_y_pred,
    mean_y_pred + std_y_pred,
    color="tab:blue",
    alpha=0.2,
)
ax2.legend()

ax2.set_ylabel("PWV (mm)")
ax2.set_xlabel("mjd")
ax2.set_title("Time-Zoom on Fit PWV with Gaussian process")
ax2.set_xlim(mjd_zoom_start,mjd_zoom_stop)
ax2.text(0.1, 0.95, txtstr_kernel, transform=ax2.transAxes, fontsize=16,verticalalignment='top', bbox=props)



figname =f"{pathfigs}/pwv_fitgp_merra2"+figtype
fig.savefig(figname)
plt.show()


## Residuals

In [ ]:
mean_y_pred_data, std_y_pred_data = gaussian_process.predict(X, return_std=True)

In [ ]:
residuals = y -  mean_y_pred_data
residuals_normalised = residuals/std_y_pred_data 

In [ ]:
fig = plt.figure(figsize=(12,4),layout="constrained")
#gs = GridSpec(2, 1,figure=fig)
gs = GridSpec(1, 1,figure=fig)
ax1 = fig.add_subplot(gs[0])
#ax2 = fig.add_subplot(gs[1])
ax1.plot(x,residuals,'-',color="k",ms=5,linestyle="dotted")
ax1.plot(x,residuals,'o',color="r",ms=5,label="M2 Measurements")
#ax1.fill_between(x,residuals, - std_y_pred_data, + std_y_pred_data,  color="tab:blue",alpha=0.2)
ax1.legend()

ax1.set_ylabel("$\Delta$ PWV (mm)")
ax1.set_xlabel("mjd")
ax1.set_title("Residuals after Fit PWV with Gaussian process ")
ax1.text(0.1, 0.95, txtstr_kernel, transform=ax1.transAxes, fontsize=16,verticalalignment='top', bbox=props)

#ax2.plot(x,residuals,'-o',color="k",ms=5,linestyle="dotted", label="M2 Measurements")
#ax2.fill_between(x,residuals, - std_y_pred_data, + std_y_pred_data,  color="tab:blue",alpha=0.2)

#ax2.legend()

#ax2.set_ylabel("PWV (mm)")
#ax2.set_xlabel("mjd")
#ax2.set_title("Time-Zoom on Fit PWV with Gaussian process")
#ax2.set_xlim(mjd_zoom_start,mjd_zoom_stop)
#ax2.text(0.1, 0.95, txtstr_kernel, transform=ax2.transAxes, fontsize=16,verticalalignment='top', bbox=props)

figname =f"{pathfigs}/pwvres_fitgp_merra2"+figtype
fig.savefig(figname)
plt.show()
